# TITLE

## IMPORTS

In [1]:
#there will be more packages added as needed
#standard datascience packages
import pandas as pd
import numpy as np
#packages for reading geojson file
import json
import os
#package for reading sql lite database
import sqlite3
#package for map making
import folium
#package for accurate date/time manipulations
import datetime
#note this package is NOT native to Anaconda, follow tabula
#documentation for steps on installation
#package for PDF scraping
import tabula


## DATASETS

### GEOJSON

This GeoJSON file contains the boundries for all California Counties as drawn by CALFIRE's Fire and Resource Assessment Program (FRAP). This dataset will help in determining boundries for map vizualizations.

**Data Source:** https://data.ca.gov/dataset/california-counties

In [2]:
ca_counties = json.load(open('California_Counties.geojson'))
ca_counties['features'][0]['properties']

{'OBJECTID': 1,
 'COUNTY_NAME': 'Alameda County',
 'COUNTY_ABBREV': 'ALA',
 'COUNTY_NUM': 1,
 'COUNTY_CODE': '01',
 'COUNTY_FIPS': '001'}

### SQL Database

SQL Lite database obtained from Kaggle contains 1.88 million wildfires throughout the US and US Territories. For the purposes of our analysis we will only use fires that are located in California. This dataset contains a myriad of information such as different ID types, latitude and longitude measurements, fire size, discovery date, and containment date. This dataset will be used in conjunction with the GeoJSON file to make the map vizualizations

**Data Source:** https://www.kaggle.com/rtatman/188-million-us-wildfires

In [3]:
conn = sqlite3.connect("fire_sql.sqlite")
fires = pd.read_sql_query("SELECT * FROM fires", conn)
fires.head()

OBJECTID  FOD_ID      FPA_ID SOURCE_SYSTEM_TYPE SOURCE_SYSTEM  \
0         1       1  FS-1418826                FED   FS-FIRESTAT   
1         2       2  FS-1418827                FED   FS-FIRESTAT   
2         3       3  FS-1418835                FED   FS-FIRESTAT   
3         4       4  FS-1418845                FED   FS-FIRESTAT   
4         5       5  FS-1418847                FED   FS-FIRESTAT   

  NWCG_REPORTING_AGENCY NWCG_REPORTING_UNIT_ID  NWCG_REPORTING_UNIT_NAME  \
0                    FS                USCAPNF    Plumas National Forest   
1                    FS                USCAENF  Eldorado National Forest   
2                    FS                USCAENF  Eldorado National Forest   
3                    FS                USCAENF  Eldorado National Forest   
4                    FS                USCAENF  Eldorado National Forest   

  SOURCE_REPORTING_UNIT SOURCE_REPORTING_UNIT_NAME  \
0                  0511     Plumas National Forest   
1                  0503   Eldorado National Forest   
2                  0503   Eldorado National Forest   
3                  0503   Eldorado National Forest   
4                  0503   Eldorado National Forest   

                         ...                         FIRE_SIZE_CLASS  \
0                        ...                                       A   
1                        ...                                       A   
2                        ...                                       A   
3                        ...                                       A   
4                        ...                                       A   

    LATITUDE   LONGITUDE OWNER_CODE       OWNER_DESCR STATE COUNTY FIPS_CODE  \
0  40.036944 -121.005833        5.0              USFS    CA     63       063   
1  38.933056 -120.404444        5.0              USFS    CA     61       061   
2  38.984167 -120.735556       13.0  STATE OR PRIVATE    CA     17       017   
3  38.559167 -119.913333        5.0              USFS    CA      3       003   
4  38.559167 -119.933056        5.0              USFS    CA      3       003   

   FIPS_NAME                                              Shape  
0     Plumas  b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...  
1     Placer  b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...  
2  El Dorado  b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...  
3     Alpine  b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...  
4     Alpine  b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...  

[5 rows x 39 columns]

### PDF SCRAPING

Unfortunately there is not any good datasets for wildfires that are in a nice format. For this reason, we have decided to collect the data ourselves the messy way from... PDF files. The PDF format was not meant to be read by a computer but by using the `Tabula` package, an open source PDF scraper, we are able to read pdf tables directly to pandas DataFrames. The tool was far from perfect, combined with inconsistencies in government data collection, required each pdf to be scraped individually page by page. To further complicate things, the DataFrames read are very messy. Data cleaning/data wrangling will be covered in the next section.

**Link to example pdf:** http://cdfdata.fire.ca.gov/pub/cdf/images/incidentstatsevents_272.pdf

**Data Source:** http://cdfdata.fire.ca.gov/incidents/incidents_statsevents

In [4]:
#we will scrape some data directly from pdf using tabula open source pdf reader.

#variable format largest_[year]_[page]
#2012 link is broken
# (do this later) TODO: drop anything that is missing a date
raw_fire_array = []
#2000 2 pages
largest_2000_1 = tabula.read_pdf('largest_2000.pdf', pages=1, area=[60, 105, 1004, 800])
raw_fire_array.append(largest_2000_1)
largest_2000_2 = tabula.read_pdf('largest_2000.pdf', pages=2, area=[60, 51, 1004, 650])
raw_fire_array.append(largest_2000_2)
#2001 2 pages
largest_2001_1 = tabula.read_pdf('largest_2001.pdf', pages=1, area=[100, 88, 952, 792])
raw_fire_array.append(largest_2001_1)
largest_2001_2 = tabula.read_pdf('largest_2001.pdf', pages=2, area=[100, 46, 952, 623])
raw_fire_array.append(largest_2001_2)
#2002 3 pages
largest_2002_1 = tabula.read_pdf('largest_2002.pdf', pages=1, area=[76, 104, 1010, 797])
raw_fire_array.append(largest_2002_1)
largest_2002_2 = tabula.read_pdf('largest_2002.pdf', pages=2, area=[76, 50, 1010, 786])
raw_fire_array.append(largest_2002_2)
largest_2002_3 = tabula.read_pdf('largest_2002.pdf', pages=3, area=[76, 50, 1010, 681])
raw_fire_array.append(largest_2002_3)
#2003 3 pages
largest_2003_1 = tabula.read_pdf('largest_2003.pdf', pages=1, area=[58, 81, 1015, 787])
raw_fire_array.append(largest_2003_1)
largest_2003_2 = tabula.read_pdf('largest_2003.pdf', pages=2, area=[58, 33, 1015, 784])
raw_fire_array.append(largest_2003_2)
largest_2003_3 = tabula.read_pdf('largest_2003.pdf', pages=3, area=[58, 33, 1015, 169])
raw_fire_array.append(largest_2003_3)
#2004 2 pages
largest_2004_1 = tabula.read_pdf('largest_2004.pdf', pages=1, area=[81, 125, 1017, 797])
raw_fire_array.append(largest_2004_1)
largest_2004_2 = tabula.read_pdf('largest_2004.pdf', pages=2, area=[81, 76, 1017, 646])
raw_fire_array.append(largest_2004_2)
#2005 3 pages
largest_2005_1 = tabula.read_pdf('largest_2005.pdf', pages=1, area=[110, 86, 958, 793])
raw_fire_array.append(largest_2005_1)
largest_2005_2 = tabula.read_pdf('largest_2005.pdf', pages=2, area=[110, 86, 958, 793])
raw_fire_array.append(largest_2005_2)
largest_2005_3 = tabula.read_pdf('largest_2005.pdf', pages=3, area=[110, 86, 958, 385])
raw_fire_array.append(largest_2005_3)
#2006 4 pages
largest_2006_1 = tabula.read_pdf('largest_2006.pdf', pages=1, area=[64, 90, 1005, 806])
raw_fire_array.append(largest_2006_1)
largest_2006_2 = tabula.read_pdf('largest_2006.pdf', pages=2, area=[64, 90, 1005, 806])
raw_fire_array.append(largest_2006_2)
largest_2006_3 = tabula.read_pdf('largest_2006.pdf', pages=3, area=[64, 90, 1005, 806])
raw_fire_array.append(largest_2006_3)
largest_2006_4 = tabula.read_pdf('largest_2006.pdf', pages=4, area=[64, 90, 1005, 633])
raw_fire_array.append(largest_2006_4)
#2007 3 pages
largest_2007_1 = tabula.read_pdf('largest_2007.pdf', pages=1, area=[97, 86, 973, 793])
raw_fire_array.append(largest_2007_1)
largest_2007_2 = tabula.read_pdf('largest_2007.pdf', pages=2, area=[97, 86, 973, 793])
raw_fire_array.append(largest_2007_2)
largest_2007_3 = tabula.read_pdf('largest_2007.pdf', pages=3, area=[97, 86, 973, 634])
raw_fire_array.append(largest_2007_3)
#2008 6 pages
largest_2008_1 = tabula.read_pdf('largest_2008.pdf', pages=1, area=[69, 86, 1034, 804])
raw_fire_array.append(largest_2008_1)
largest_2008_2 = tabula.read_pdf('largest_2008.pdf', pages=2, area=[69, 86, 1034, 804])
raw_fire_array.append(largest_2008_2)
largest_2008_3 = tabula.read_pdf('largest_2008.pdf', pages=3, area=[69, 86, 1034, 804])
raw_fire_array.append(largest_2008_3)
largest_2008_4 = tabula.read_pdf('largest_2008.pdf', pages=4, area=[69, 86, 1034, 804])
raw_fire_array.append(largest_2008_4)
largest_2008_5 = tabula.read_pdf('largest_2008.pdf', pages=5, area=[69, 86, 1034, 804])
raw_fire_array.append(largest_2008_5)
largest_2008_6 = tabula.read_pdf('largest_2008.pdf', pages=1, area=[69, 86, 1034, 260])
raw_fire_array.append(largest_2008_6)
#2009 3 pages
largest_2009_1 = tabula.read_pdf('largest_2009.pdf', pages=1, area=[72, 199, 1052, 780])
raw_fire_array.append(largest_2009_1)
largest_2009_2 = tabula.read_pdf('largest_2009.pdf', pages=2, area=[72, 89, 1052, 788])
raw_fire_array.append(largest_2009_2)
largest_2009_3 = tabula.read_pdf('largest_2009.pdf', pages=3, area=[72, 89, 1052, 390])
raw_fire_array.append(largest_2009_3)
#2010 2 pages
largest_2010_1 = tabula.read_pdf('largest_2010.pdf', pages=1, area=[69, 435, 1052, 780])
raw_fire_array.append(largest_2010_1)
largest_2010_2 = tabula.read_pdf('largest_2010.pdf', pages=2, area=[69, 88, 1052, 738])
raw_fire_array.append(largest_2010_2)
#2011 2 pages
largest_2011_1 = tabula.read_pdf('largest_2011.pdf', pages=1, area=[86, 117, 1037, 815])
raw_fire_array.append(largest_2011_1)
largest_2011_2 = tabula.read_pdf('largest_2011.pdf', pages=2, area=[86, 121, 1037, 731])
raw_fire_array.append(largest_2011_2)
#2013 2 pages (remember 2012 link is broken)
largest_2013_1 = tabula.read_pdf('largest_2013.pdf', pages=1, area=[102, 90, 960, 785])
raw_fire_array.append(largest_2013_1)
largest_2013_2 = tabula.read_pdf('largest_2013.pdf', pages=2, area=[102, 50, 960, 417])
raw_fire_array.append(largest_2013_2)
#2014 2 pages
largest_2014_1 = tabula.read_pdf('largest_2014.pdf', pages=1, area=[220, 155, 1658, 1332])
raw_fire_array.append(largest_2014_1)
largest_2014_2 = tabula.read_pdf('largest_2014.pdf', pages=2, area=[220, 155, 1658, 818])
raw_fire_array.append(largest_2014_2)
#2015 3 pages
largest_2015_1 = tabula.read_pdf('largest_2015.pdf', pages=1, area=[43, 93, 1058, 717])
raw_fire_array.append(largest_2015_1)
largest_2015_2 = tabula.read_pdf('largest_2015.pdf', pages=2, area=[43, 125, 1058, 670])
raw_fire_array.append(largest_2015_2)
largest_2015_3 = tabula.read_pdf('largest_2015.pdf', pages=3, area=[43, 107, 1058, 346])
raw_fire_array.append(largest_2015_3)
#2016 3 pages
largest_2016_1 = tabula.read_pdf('largest_2016.pdf', pages=1, area=[38, 162, 1061, 792])
raw_fire_array.append(largest_2016_1)
largest_2016_2 = tabula.read_pdf('largest_2016.pdf', pages=2, area=[38, 177, 1061, 786])
raw_fire_array.append(largest_2016_2)
largest_2016_3 = tabula.read_pdf('largest_2016.pdf', pages=1, area=[38, 100, 1061, 372])
raw_fire_array.append(largest_2016_3)

#45 total pages!

#### Messy DataFrame Example

The table is practically unreadable, many problems to fix such as combined columns, NaN/missing columns, useless rows, and incorrect/missing column names.

In [5]:
largest_2004_1.head()

Unnamed: 0  Unnamed: 1               Unnamed: 2  \
0            INC. # COUNTY         NaN                FIRE NAME   
1  BDF-3917 SAN BERNARDINO         NaN          UPPER SANTA ANA   
2        SHF-000443 TEHAMA         NaN    SHF ASSIST (HAMILTON)   
3     RRU-032913 RIVERSIDE         NaN            PLEASURE FIRE   
4     MVU-003337 SAN DIEGO         NaN  ASSIST #31 (INDIA FIRE)   

                     DATE  Unnamed: 4     ACRES BURNED  VEG. Unnamed: 7  \
0         START CONT. DPA         NaN  CDF OTHER TOTAL  TYPE      CAUSE   
1  03/25/04 03/26/04 USFS         NaN          350 350    BT     ESCAPE   
2  04/09/04 04/12/04 USFS         NaN          301 301     T      HUMAN   
3   04/25/04 04/26/04 CDF         NaN      2,464 2,464    GB    VEHICLE   
4  05/02/04 05/03/04 MIL.         NaN      2,040 2,040    BT         UI   

    STRUCTURESFATALITIES  
0  DEST. DAM. FIRE CIVIL  
1                1 0 0 0  
2                0 0 0 0  
3                9 0 0 0  
4                0 0 0 0

### POSSIBLE NEW AVENUES
National overall statistics on fires and economic losses 

https://www.usfa.fema.gov/data/statistics/ 


Link with federal government resources like firestations (most data are restricted)

https://hifld-geoplatform.opendata.arcgis.com/


## Data Cleaning/Wrangling

### UNIVERSAL VARIABLES

In [6]:
#Row names for cleaned tables
ROWS_N = ['INCIDENT #', 'COUNTY/UNIT', 'FIRE_NAME', 'START_DATE', 
          'CONT_DATE', 'ORIGIN_DPT', 'BURNED_TOTAL', 'VEG_TYPE', 
          'CAUSE', 'STRUCT_DEST', 'STRUCT_DAM', 'FATALITY_FIRE', 
          'FATALITY_CIVIL']
STRUCT_ROW = ['STRUCT_DEST', 'STRUCT_DAM']
FATAL_ROW = ['FATALITY_FIRE', 'FATALITY_CIVIL']

### HELPER FUNCTIONS

In [7]:
#TODO: Fix 2000 - 2002 Years to use robust_col_sep
"""
NOTE: this only works for fatality and structural columns

column_separator(df, col, new_names)

Separates one column into two (if values are separated by spaces)

params:
df : DataFrame to change
col : column name to split
new_names : array of 2 new names for columns

returns:
pandas DataFrame with changes applied
"""
def column_separator(df, col, new_names):
    column = df[col].astype(str).apply(lambda x : x.split(' '))
    left_arr = []
    right_arr = []
    #uncomment below to see actual array values
    #display(column)
    
    #iterate through values to separate them
    for val in column:
        #edge case array is len 1
        if len(val) == 1:
            left_arr.append(np.nan)
            right_arr.append(np.nan)
        #normal case array len 2
        elif len(val) == 2:
            left_arr.append(val[0])
            right_arr.append(val[1])
        #edge case array is bigger than expected
        else:
            left_arr.append('error')
            right_arr.append('error')
    #now make new column names and remove the old one
    #display(left_arr)
    #display(right_arr)
    ans_df = df.drop(col, axis=1)
    ans_df[new_names[0]] = left_arr
    ans_df[new_names[1]] = right_arr
    return ans_df
"""
robust_col_sep(df, iloc, new_names, front=True)

Separates one column into two columns, values must be separated by spaces; drops the original column at iloc index

params:

df : the dataframe to modify
index : the index of the column as given by iloc indexing (int)
new_names : the new names for the two new columns created (array size 2)
front : determines whether to separate the value from the front or from the back (boolean, optional):

Example:
Consider the following string:
Apple Cinnamon Bannana

front = True will give
[Apple] [Cinnamon Bannana]

front = False will give
[Apple Cinnamon] [Bannana]

returns:
pandas DataFrame with changes applied
"""

def robust_col_sep(df, index, new_names, front=True):
    column = df.iloc[:, index].astype(str).apply(lambda x : x.split(' '))
    left_arr = []
    right_arr = []
    
    #display(column)
    #iterate through column vals
    for val in column:
        if front:
            left_arr.append(val[0])
            right_arr.append(' '.join(val[1:]))
        #if not front
        else:
            left_arr.append(' '.join(val[:-1]))
            right_arr.append(val[-1])
    #make new cols and drop old
    ans_df = df.drop(df.columns[index], axis=1)
    #add right column first
    #display(len(left_arr))
    #display(len(right_arr))
    #display(len(df.index))
    ans_df.insert(index, new_names[1], right_arr)
    ans_df.insert(index, new_names[0], left_arr)
    return ans_df

############################################################################
#                           APPLY HELPER FUNCTIONS                         #
############################################################################

'''
APPLY HELPER FUNCTION (used in Series.apply() only)

Takes in an array with year and DOY (Day Of Year) and converts it to a datetime object.
This function is needed due to odd dating format in SQL table
'''
def datetime_convert_sql(year_day_array):
    year = int(year_day_array[0])
    #issue with nan values not converting to int values
    try:
        day = int(year_day_array[1].split('.')[0])
        ans = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day-1)
    except ValueError:
        ans = np.nan
    return ans
'''
APPLY HELPER FUNCTION (used in Series.apply() only)

Takes in a string in the format MM/DD/YY and converts it to a datetime object
This function is needed due to YY format and NaN error handling
'''
def datetime_convert_pdf(date):
    date_arr = date.split('/')
    if len(date_arr) == 3:
        return datetime.datetime(int('20'+date_arr[2].split('.')[0]), int(date_arr[0]), int(date_arr[1]))
    else:
        return np.nan
'''
APPLY HELPER FUNCTION (used in Series.apply() only)

Takes in a string value and converts to integer value
This function is needed for NaN value handleing to join PDF table and SQL table
'''
def int_convert(val):
    try:
        ans = int(val)
    except:
        ans = -1
    return ans

### SQL TABLE CLEANING

The SQL table is largely clean already, we will just drop a few extra columns and convert the strange dates into datetime objects as well as clean up the `FIRE_SIZE` column into `int` format.

In [8]:
#TODO: CLEAN UP THIS SPAGHETTI CODE MAN!
fires = fires[fires['STATE'] == 'CA']
fires = fires.drop(columns=fires.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 16, 17, 18, 24, 25, 32, 33, 36, 37, 38]])
#preparing for date conversion
fires['START_DATE'] = fires[['FIRE_YEAR', 'DISCOVERY_DOY']].astype(str).apply(lambda x: ' '.join(x).split(' '), axis=1)
fires['END_DATE'] = fires[['FIRE_YEAR', 'CONT_DOY']].astype(str).apply(lambda x: ' '.join(x).split(' '), axis=1)
#date conversion
fires['START_DATE'] = fires.START_DATE.apply(datetime_convert_sql)
fires['END_DATE'] = fires.END_DATE.apply(datetime_convert_sql)
fires = fires.drop(['FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 
                    'CONT_DOY'], axis=1)
fires.FIRE_SIZE = fires.FIRE_SIZE.astype(int)
fires.columns

Index(['OBJECTID', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_NAME',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'CONT_TIME', 'FIRE_SIZE',
       'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'STATE', 'COUNTY',
       'START_DATE', 'END_DATE'],
      dtype='object')

### PDF TABLE CLEANING

Mostly cleaned datasets anyways, largest issue is the combination of `County/Unit` and `Fire Name`. Normally, I would fix these issues by splitting on spaces (' ') but some departments and fire names contain more than one word so this method will not work. There are two ideas I have to solve this issues but it requires to add all the datasets in first

This issue does not happen with all years which means some will have correct separation between `County/Unit` and `Fire Name` which means we can do one of two:
1. Once we have all datasets combined, we can make a list of all counties/units and search for them in each grouped string
2. There seems to be some correlation between the first three letters of Incident # and the county/unit, we can map the county from the Incident # (Incident # is fairly complete and easy to extract).

In [9]:
#TODO: Update 2000 - 2002 years to new cleaning procedures

#2000
#page 1
#columns are rows, add correct columns, superfluous col
clean_2000_1 = largest_2000_1.append(largest_2000_1.columns.to_series(), ignore_index=True)
clean_2000_1 = clean_2000_1.drop(['100%', 'Unnamed: 7', '400'], axis=1)
clean_2000_1.columns = ROWS_N
#page 2
#columns are a row, add correct cols, structural cols connected
clean_2000_2 = largest_2000_2.dropna(subset=['08/27/00'])
clean_2000_2 = clean_2000_2.drop(['Unnamed: 13', 'Unnamed: 7', '8,084'], axis=1)
clean_2000_2 = clean_2000_2.append(clean_2000_2.columns.to_series(), ignore_index=True)
clean_2000_2['x'] = np.nan
clean_2000_2['y'] = np.nan
clean_2000_2.columns = ROWS_N
clean_2000_2['INCIDENT #'] = clean_2000_2['INCIDENT #'].apply(lambda x: x.split(' ')[0])

#2001
#page 1
# columns and row 0 useless, col 11 useless
# structure and fatality rows are stuck
# drop acres burned broken (except total)
clean_2001_1 = largest_2001_1.drop(0).drop(['Unnamed: 11', '%', 'ACRES BURNED'], axis=1)
clean_2001_1 = column_separator(clean_2001_1, 'STRUCTURES', STRUCT_ROW)
clean_2001_1 = column_separator(clean_2001_1, 'FATALITIES', FATAL_ROW)
clean_2001_1.columns = ROWS_N
clean_2001_1
# page 2
#columns and row 0 useless, % col 3, 13 useless
#drop nan start date
#connected columns : unit/name structures fatalities
# TODO: UNIT AND FIRE NAME
clean_2001_2 = largest_2001_2.drop(0).drop(['%', 'Unnamed: 3', 'ACRES BURNED', 'Unnamed: 13'], axis = 1)
clean_2001_2 = clean_2001_2.dropna(subset=['DATE'])
clean_2001_2.insert(2, 'placeholder', -1)
clean_2001_2 = column_separator(clean_2001_2, 'STRUCTURES', STRUCT_ROW)
clean_2001_2 = column_separator(clean_2001_2, 'FATALITIES', FATAL_ROW)
clean_2001_2.columns = ROWS_N
clean_2001_2

#2002
#page 1
#column is row
#connected columns: unit/name
# TODO: UNIT AND FIRE NAME
clean_2002_1 = largest_2002_1.drop(['100%', '320', 'Unnamed: 8'], axis=1)
clean_2002_1 = clean_2002_1.append(clean_2002_1.columns.to_series(), ignore_index=True)
clean_2002_1.columns = ROWS_N
clean_2002_1
#page 2
#column is row
clean_2002_2 = largest_2002_2.drop(['100%', 'Unnamed: 7', '670'], axis=1)
clean_2002_2 = clean_2002_2.append(clean_2002_2.columns.to_series(), ignore_index=True)
clean_2002_2.columns = ROWS_N
clean_2002_2
#page 3
#column is row
#drop na dates
#combined columns: name/date structure fatality
clean_2002_3 = largest_2002_3.drop(['Unnamed: 6', '1,200'], axis=1)
clean_2002_3 = column_separator(clean_2002_3, 'Unnamed: 11', STRUCT_ROW)
clean_2002_3 = column_separator(clean_2002_3, 'Unnamed: 12', FATAL_ROW)
clean_2002_3 = clean_2002_3.dropna(subset=['Mountain 09/09/02'])
clean_2002_3.columns = ROWS_N
#procedure for separating firename and date
clean_2002_3.START_DATE = clean_2002_3.FIRE_NAME.apply(lambda x: x.split(' ')[-1])
clean_2002_3.FIRE_NAME = clean_2002_3.FIRE_NAME.apply(lambda x: ' '.join(x.split(' ')[:-1]))

#END TODO

#2003
#page 1
largest_2003_1
clean_2003_1 = robust_col_sep(largest_2003_1, 0, ['apples', 'bannanas'])
clean_2003_1 = robust_col_sep(clean_2003_1, 4, ['oranges', 'strawberries'])
clean_2003_1 = clean_2003_1.drop(clean_2003_1.columns[[2, 6, 8, 9, 13]], axis=1)
clean_2003_1 = clean_2003_1.drop(0)
clean_2003_1 = robust_col_sep(clean_2003_1, 9, ['mangos', 'peaches'])
clean_2003_1 = robust_col_sep(clean_2003_1, 11, ['grapefruit', 'lemons'])
#clean_2003_1 = clean_2003_1.drop(clean_2003_1.columns[[1]], axis=1)
clean_2003_1.columns = ROWS_N
#page 2
clean_2003_2 = robust_col_sep(largest_2003_2, 0, ['1', '2']) 
clean_2003_2 = robust_col_sep(clean_2003_2, 4, ['3', '4'])
clean_2003_2 = robust_col_sep(clean_2003_2, 13, ['13', '14'])
clean_2003_2 = robust_col_sep(clean_2003_2, 15, ['15', '16'])
clean_2003_2 = robust_col_sep(clean_2003_2, 11, ['11', '12'])
clean_2003_2 = clean_2003_2.drop(0)
clean_2003_2 = clean_2003_2.drop(columns=clean_2003_2.columns[[3, 6, 8, 9, 13]])
clean_2003_2.columns = ROWS_N
#page 3
clean_2003_3 = largest_2003_3.drop(range(7,12))
clean_2003_3 = robust_col_sep(clean_2003_3, 0, [0, 1])
clean_2003_3 = robust_col_sep(clean_2003_3, 5, [5, 6])
clean_2003_3 = robust_col_sep(clean_2003_3, 9, [9, 10])
clean_2003_3 = robust_col_sep(clean_2003_3, 14, [14, 15])
clean_2003_3 = robust_col_sep(clean_2003_3, 16, [16, 17])
clean_2003_3 = clean_2003_3.drop(columns=clean_2003_3.columns[[2, 4, 8, 10, 11]])
clean_2003_3 = clean_2003_3.drop(0)
clean_2003_3.columns = ROWS_N

#2004
#page1
clean_2004_1 = robust_col_sep(largest_2004_1, 0, ['0','1'])
clean_2004_1 = robust_col_sep(clean_2004_1, 4, ['4','5'])
clean_2004_1 = robust_col_sep(clean_2004_1, 5, ['5','6'])
clean_2004_1 = robust_col_sep(clean_2004_1, 8, ['8','9'], front=False)
#special case
abomination = clean_2004_1.STRUCTURESFATALITIES
clean_2004_1['12'] = abomination.apply(lambda x: x.split(' ')[0])
clean_2004_1['13'] = abomination.apply(lambda x: x.split(' ')[1])
clean_2004_1['14'] = abomination.apply(lambda x: x.split(' ')[2])
clean_2004_1['15'] = abomination.apply(lambda x: x.split(' ')[3])
clean_2004_1 = clean_2004_1.drop(['STRUCTURESFATALITIES'], axis=1)
# back to regular programing
clean_2004_1 = clean_2004_1.drop(0)
clean_2004_1 = clean_2004_1.drop(columns=clean_2004_1.columns[[2, 7, 8]])
clean_2004_1.columns = ROWS_N
#page2
clean_2004_2 = robust_col_sep(largest_2004_2, 0, ['0', '1'])
clean_2004_2 = robust_col_sep(clean_2004_2, 4, ['4', '5'])
clean_2004_2 = robust_col_sep(clean_2004_2, 5, ['5', '6'])
clean_2004_2 = robust_col_sep(clean_2004_2, 8, ['8', '9'])
#another abomination
abomination = clean_2004_2.STRUCTURESFATALITIE
clean_2004_2['12'] = abomination.apply(lambda x: x.split(' ')[0])
clean_2004_2['13'] = abomination.apply(lambda x: x.split(' ')[1])
clean_2004_2['14'] = abomination.apply(lambda x: x.split(' ')[2])
clean_2004_2['15'] = 0
clean_2004_2 = clean_2004_2.drop(['STRUCTURESFATALITIE'], axis=1)
#end special case
clean_2004_2 = clean_2004_2.drop([0, 38])
clean_2004_2 = clean_2004_2.drop(columns=clean_2004_2.columns[[3, 7, 8]])
clean_2004_2.columns = ROWS_N
#37 pages left

#2005

# 2015
# Clean the 2015_page_1
clean_2015_1 = largest_2015_1
clean_2015_1 = clean_2015_1.drop(0)
clean_2015_1.columns = ROWS_N
clean_2015_1['ORIGIN_DPT'] = clean_2015_1['BURNED_TOTAL']
clean_2015_1['BURNED_TOTAL'] = clean_2015_1['VEG_TYPE']
clean_2015_1['VEG_TYPE'] = clean_2015_1['STRUCT_DEST']
clean_2015_1['CAUSE'] = clean_2015_1['STRUCT_DAM']
clean_2015_1 = column_separator(clean_2015_1, "FATALITY_FIRE", STRUCT_ROW)
clean_2015_1['FATALITY_FIRE'] = np.nan
# Reorder the columns
clean_2015_1 = clean_2015_1.reindex(columns=ROWS_N)
# Refill missing data (Some values lost during column separation)
clean_2015_1.iloc[3, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 2
clean_2015_1.iloc[11, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 16
clean_2015_1.iloc[13, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 7
clean_2015_1.iloc[15, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 27
clean_2015_1.iloc[17, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 1
clean_2015_1.iloc[18, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 1
clean_2015_1.iloc[20, clean_2015_1.columns.get_loc('STRUCT_DEST')] = 965

# Clean the 2015_page_2 
clean_2015_2 = largest_2015_2.drop(0)
clean_2015_2.columns = ROWS_N
clean_2015_2['ORIGIN_DPT'] = clean_2015_2['BURNED_TOTAL']
clean_2015_2['BURNED_TOTAL'] = clean_2015_2['VEG_TYPE']
clean_2015_2['VEG_TYPE'] = clean_2015_2['STRUCT_DEST']
clean_2015_2['CAUSE'] = clean_2015_2['STRUCT_DAM']
clean_2015_2 = column_separator(clean_2015_2, "FATALITY_FIRE", STRUCT_ROW)
clean_2015_2['FATALITY_FIRE'] = clean_2015_2['FATALITY_CIVIL']
# Reorder the columns
clean_2015_2 = clean_2015_2.reindex(columns=ROWS_N)
# Crosschecking dataframe data manually
clean_2015_2['FATALITY_CIVIL'] = np.nan
clean_2015_2.iloc[6, clean_2015_2.columns.get_loc('STRUCT_DEST')] = 4
clean_2015_2.iloc[8, clean_2015_2.columns.get_loc('STRUCT_DAM')] = 1
# Fill columns 14-18 with right values for STRUCT_DEST column
clean_2015_2.iloc[14:19, clean_2015_2.columns.get_loc('STRUCT_DEST')] = largest_2015_2['Structures'][15:20]

# Clean the 2015_page_3
# Note do we keep TOTAL?
clean_2015_3 = largest_2015_3.drop(0)
# Drop the total row
clean_2015_3 = clean_2015_3.drop(8)
clean_2015_3.columns = ROWS_N
clean_2015_3['ORIGIN_DPT'] = clean_2015_3['BURNED_TOTAL']
clean_2015_3['BURNED_TOTAL'] = clean_2015_3['VEG_TYPE']
clean_2015_3['VEG_TYPE'] = clean_2015_3['STRUCT_DEST']
clean_2015_3['CAUSE'] = clean_2015_3['STRUCT_DAM']
clean_2015_3 = column_separator(clean_2015_3, "FATALITY_FIRE", STRUCT_ROW)
clean_2015_3['FATALITY_FIRE'] = np.nan
# Reorder the columns
clean_2015_3 = clean_2015_3.reindex(columns=ROWS_N)
clean_2015_3.iloc[1, clean_2015_3.columns.get_loc('STRUCT_DEST')] = 3



## DATACLEANING TUTORIAL

The PDF data cleaning will consist mainly of 4 key procedures:
1. Separating columns that are 'stuck' together using `robust_col_sep`
    - Note that `COUNTY/UNIT` and `FIRE_NAME` columns can NOT be separated using `robust_col_sep`


2. Dropping any columns with acres burned that is NOT `BURNED_TOTAL`
    - The two types of of burned columns to delete are `CDF` and `Other`, these names are usually found in row 0
    

3. Dropping null columns and useless rows
    - For dropping rows, there may be a 'totals' row or some text rows at the end of the DataFrame, remove these too
    

4. Setting correct column names using `df.columns = ROWS_N`
    - If you get a length mismatch error, make sure all columns are separated 
    - If `COUNTY/UNIT` and `FIRE_NAME` are connected, keep the NaN column next to the combined column in order to prevent a length mismatch error. Make sure the order of the rows makes sense! 

### EXAMPLES

### EXAMPLE 1
For the first example I will use the `largest_2003_3` PDF. We want out final prouct to look like this:

In [10]:
display(clean_2003_3)

INCIDENT # COUNTY/UNIT FIRE_NAME START_DATE CONT_DATE ORIGIN_DPT  \
1    MVU-8151   San Diego      Mine   10/26/03  10/28/03        CDF   
2  RRU-082809   Riverside  Mountain   10/26/03  10/29/03        CDF   
3    HUU-5720    Humboldt       Tuk   10/26/03  10/30/03        NPS   
4  MVU-008143   San Diego  Paradise   10/26/03  11/06/03        CDF   
5   SHU-11176      Shasta  Whitmore   10/27/03  10/30/03        CDF   
6    LMU-4234       Modoc      Sand   11/28/03  11/29/03       USFS   

  BURNED_TOTAL VEG_TYPE         CAUSE STRUCT_DEST STRUCT_DAM FATALITY_FIRE  \
1       46,000        B         Arson           0          0             0   
2       10,000       GB    Equip. Use          61          0             0   
3          315        T           NaN           0          0             0   
4       56,700        B    Equip. Use         223         10             0   
5        1,200        T  Undetermined           0          0             0   
6          300       BT        Escape           0          0             0   

  FATALITY_CIVIL  
1              0  
2              0  
3              0  
4              2  
5              0  
6              0

But the dataframe starts like this:

In [11]:
display(largest_2003_3)

Unnamed: 0  Unnamed: 1  \
0                            INCIDENT # COUNTY / UNIT         NaN   
1                                  MVU-8151 San Diego         NaN   
2                                RRU-082809 Riverside         NaN   
3                                   HUU-5720 Humboldt         NaN   
4                                MVU-008143 San Diego         NaN   
5                                    SHU-11176 Shasta         NaN   
6                                      LMU-4234 Modoc         NaN   
7                                                 NaN         NaN   
8   *Shaded rows represent fires for which fire re...         NaN   
9   This large fire list includes fires 300 acres ...         NaN   
10  by the CDF unit where the fire occurred. Often...         NaN   
11  activity within their jurisdiction.  Informati...         NaN   

         Unnamed: 2 Unnamed: 3               DATE  ORIGIN Unnamed: 6  \
0               NaN  FIRE NAME        START CONT.     DPA        CDF   
1              Mine        NaN  10/26/03 10/28/03     CDF     46,000   
2          Mountain        NaN  10/26/03 10/29/03     CDF     10,000   
3               Tuk        NaN  10/26/03 10/30/03     NPS        NaN   
4          Paradise        NaN  10/26/03 11/06/03     CDF     56,700   
5          Whitmore        NaN  10/27/03 10/30/03     CDF      1,200   
6              Sand        NaN  11/28/03 11/29/03    USFS        NaN   
7   115 Large Fires        NaN                NaN  TOTALS    400,926   
8               NaN        NaN                NaN     NaN        NaN   
9               NaN        NaN                NaN     NaN        NaN   
10              NaN        NaN                NaN     NaN        NaN   
11              NaN        NaN                NaN     NaN        NaN   

       ACRES BURNED  Unnamed: 8  VEG.   Unnamed: 10  STRUCTURES  FATALITIES  
0       OTHER TOTAL         NaN  TYPE         CAUSE  DEST. DAM.  FIRE CIVIL  
1            46,000         NaN     B         Arson         0 0         0 0  
2            10,000         NaN    GB    Equip. Use        61 0         0 0  
3           315 315         NaN     T           NaN         0 0         0 0  
4            56,700         NaN     B    Equip. Use      223 10         0 2  
5             1,200         NaN     T  Undetermined         0 0         0 0  
6           300 300         NaN    BT        Escape         0 0         0 0  
7   573,243 974,169         NaN   NaN           NaN   6,632 178        1 22  
8               NaN         NaN   NaN           NaN         NaN         NaN  
9               NaN         NaN   NaN           NaN         NaN         NaN  
10              NaN         NaN   NaN           NaN         NaN         NaN  
11              NaN         NaN   NaN           NaN         NaN         NaN

### 1. Separating columns that are stuck together

A look at the DataFrame reveals the following columns are stuck together:

* `INCIDENT #` & `COUTNY/UNIT`
* `START_DATE` & `CONT_DATE`
* `Other` & `BURNED_TOTAL`
* both `STRUCTURES` columns
* both `FATALITIES` columns

lets separate them

NOTE: Check the DataFrame after every call in order to get the right index for the columns becuase you add a column on each call!

NOTE2: I recommend making the new names the index that the columns will be in, this will help later!

In [12]:
clean_2003_3 = robust_col_sep(largest_2003_3, 0, [0, 1])
clean_2003_3 = robust_col_sep(clean_2003_3, 5, [5, 6])
clean_2003_3 = robust_col_sep(clean_2003_3, 9, [9, 10])
clean_2003_3 = robust_col_sep(clean_2003_3, 14, [14, 15])
clean_2003_3 = robust_col_sep(clean_2003_3, 16, [16, 17])
clean_2003_3

0                                                  1  Unnamed: 1  \
0     INCIDENT                                    # COUNTY / UNIT         NaN   
1     MVU-8151                                          San Diego         NaN   
2   RRU-082809                                          Riverside         NaN   
3     HUU-5720                                           Humboldt         NaN   
4   MVU-008143                                          San Diego         NaN   
5    SHU-11176                                             Shasta         NaN   
6     LMU-4234                                              Modoc         NaN   
7          nan                                                            NaN   
8      *Shaded  rows represent fires for which fire reports ha...         NaN   
9         This  large fire list includes fires 300 acres and g...         NaN   
10          by  the CDF unit where the fire occurred. Often th...         NaN   
11    activity  within their jurisdiction.  Information on thi...         NaN   

         Unnamed: 2 Unnamed: 3         5         6  ORIGIN Unnamed: 6  \
0               NaN  FIRE NAME     START     CONT.     DPA        CDF   
1              Mine        NaN  10/26/03  10/28/03     CDF     46,000   
2          Mountain        NaN  10/26/03  10/29/03     CDF     10,000   
3               Tuk        NaN  10/26/03  10/30/03     NPS        NaN   
4          Paradise        NaN  10/26/03  11/06/03     CDF     56,700   
5          Whitmore        NaN  10/27/03  10/30/03     CDF      1,200   
6              Sand        NaN  11/28/03  11/29/03    USFS        NaN   
7   115 Large Fires        NaN       nan            TOTALS    400,926   
8               NaN        NaN       nan               NaN        NaN   
9               NaN        NaN       nan               NaN        NaN   
10              NaN        NaN       nan               NaN        NaN   
11              NaN        NaN       nan               NaN        NaN   

          9       10  Unnamed: 8  VEG.   Unnamed: 10     14    15    16     17  
0     OTHER    TOTAL         NaN  TYPE         CAUSE  DEST.  DAM.  FIRE  CIVIL  
1    46,000                  NaN     B         Arson      0     0     0      0  
2    10,000                  NaN    GB    Equip. Use     61     0     0      0  
3       315      315         NaN     T           NaN      0     0     0      0  
4    56,700                  NaN     B    Equip. Use    223    10     0      2  
5     1,200                  NaN     T  Undetermined      0     0     0      0  
6       300      300         NaN    BT        Escape      0     0     0      0  
7   573,243  974,169         NaN   NaN           NaN  6,632   178     1     22  
8       nan                  NaN   NaN           NaN    nan         nan         
9       nan                  NaN   NaN           NaN    nan         nan         
10      nan                  NaN   NaN           NaN    nan         nan         
11      nan                  NaN   NaN           NaN    nan         nan

### 2. & 3. Dropping other burned acres, NaN columns, and useless rows

Some notes about the code:
* `df.drop` has a `columns` keyword that is the same as doing `axis=1` in order to drop columns
* the form `df.columns[[number list]]` drops the columns at that index, this helps so you dont have to write the whole name down 
* we have some text rows and a total row at the bottom we can delete
* the `range(start, end)` code snippet makes an array with the range \[start, end)

In [13]:
clean_2003_3 = clean_2003_3.drop(columns=clean_2003_3.columns[[2, 4, 8, 10, 11]])
clean_2003_3 = clean_2003_3.drop(0)
clean_2003_3 = clean_2003_3.drop(range(7,12))
clean_2003_3

0          1 Unnamed: 2         5         6 ORIGIN       9 VEG.  \
1    MVU-8151  San Diego       Mine  10/26/03  10/28/03    CDF  46,000    B   
2  RRU-082809  Riverside   Mountain  10/26/03  10/29/03    CDF  10,000   GB   
3    HUU-5720   Humboldt        Tuk  10/26/03  10/30/03    NPS     315    T   
4  MVU-008143  San Diego   Paradise  10/26/03  11/06/03    CDF  56,700    B   
5   SHU-11176     Shasta   Whitmore  10/27/03  10/30/03    CDF   1,200    T   
6    LMU-4234      Modoc       Sand  11/28/03  11/29/03   USFS     300   BT   

    Unnamed: 10   14  15 16 17  
1         Arson    0   0  0  0  
2    Equip. Use   61   0  0  0  
3           NaN    0   0  0  0  
4    Equip. Use  223  10  0  2  
5  Undetermined    0   0  0  0  
6        Escape    0   0  0  0

### 4. Add correct column names

This is probably the easiest part if everything else was done right just use `df.columns = ROWS_N`

In [14]:
clean_2003_3.columns = ROWS_N
clean_2003_3

INCIDENT # COUNTY/UNIT FIRE_NAME START_DATE CONT_DATE ORIGIN_DPT  \
1    MVU-8151   San Diego      Mine   10/26/03  10/28/03        CDF   
2  RRU-082809   Riverside  Mountain   10/26/03  10/29/03        CDF   
3    HUU-5720    Humboldt       Tuk   10/26/03  10/30/03        NPS   
4  MVU-008143   San Diego  Paradise   10/26/03  11/06/03        CDF   
5   SHU-11176      Shasta  Whitmore   10/27/03  10/30/03        CDF   
6    LMU-4234       Modoc      Sand   11/28/03  11/29/03       USFS   

  BURNED_TOTAL VEG_TYPE         CAUSE STRUCT_DEST STRUCT_DAM FATALITY_FIRE  \
1       46,000        B         Arson           0          0             0   
2       10,000       GB    Equip. Use          61          0             0   
3          315        T           NaN           0          0             0   
4       56,700        B    Equip. Use         223         10             0   
5        1,200        T  Undetermined           0          0             0   
6          300       BT        Escape           0          0             0   

  FATALITY_CIVIL  
1              0  
2              0  
3              0  
4              2  
5              0  
6              0

### EXAMPLE 2

For the second example I will use `largest_2004_1`

We want the DataFrame to look like this:

In [16]:
display(clean_2004_1)

INCIDENT #     COUNTY/UNIT                       FIRE_NAME  \
1             BDF-3917  SAN BERNARDINO                 UPPER SANTA ANA   
2           SHF-000443          TEHAMA           SHF ASSIST (HAMILTON)   
3           RRU-032913       RIVERSIDE                   PLEASURE FIRE   
4           MVU-003337       SAN DIEGO         ASSIST #31 (INDIA FIRE)   
5            RRU-35190       RIVERSIDE                           EAGLE   
6            RRU-35197       RIVERSIDE                         GAFFORD   
7            RRU-35567       RIVERSIDE                     SCHOOL INC.   
8           RRU-035517       RIVERSIDE                CERRITO FIRE 681   
9             SLU-3169   SANTA BARBARA             ASSIST #6 (CACHUMA)   
10          TUU-004886          TULARE                         FRAZIER   
11           RRU-38418       RIVERSIDE                      COTTONWOOD   
12          CA-PNF-379          PLUMAS                      COTTONWOOD   
13          TUU-005509          TULARE                           STAGE   
14            MMU-8972          MERCED                          BASALT   
15          NEU-009119            YUBA                           BEALE   
16          SCU-003038     SANTA CLARA                        PRIDMORE   
17            SBC-4032   SANTA BARBARA                         GAVIOTA   
18          TUU-006407          TULARE                          CASTLE   
19            SNF-1029          MADERA                          SOURCE   
20            SQF-1461          TULARE                            CRAG   
21            TUU-6885            KERN            KERN #11 (WEST FIRE)   
22          AEU-010202          AMADOR                     MOKE ASSIST   
23            LMU-1852          LASSEN                       SKEDADDLE   
24            BDU-6547     LOS ANGELES  ASSIST ANF (INTERSTATE06/26/04   
25          FKU-007654          FRESNO                        FKU-7654   
26  MMU-018987MARIPOSA                                          MEADOW   
27            LNP-2017          SHASTA                       BLUFF WFU   
28          BEU-002411        MONTEREY            CABEU002411 (CHULAR)   
29          TGU-003837        TUOLUMNE                        BLACK, H   
30            MVU-5269       SAN DIEGO                          WARNER   
31          RRU-055439       RIVERSIDE                        VERBENIA   
32            SCU-3829      SANTA CRUZ                   CORRAL HOLLOW   
33           FKU-08513          FRESNO                           GATOS   
34          NEU-011956          PLACER                          SUNSET   
35          SCU-003929     SANTA CLARA                          SILVER   
36            BDU-7509     LOS ANGELES                            PINE   
37            MVU-5659       SAN DIEGO                       MATAGUARY   
38          RRU-056039       RIVERSIDE                        LAKEVIEW   
39          TUU-008451            KERN                      ROCKYKRN#1   
40           RRU-57236       RIVERSIDE                  STATE (MELTON)   
41            BDU-7498     LOS ANGELES               ASSIST (FOOTHILL)   
42          FKU-008892          FRESNO                         MENDOTA   
43            SBC-5251   SANTA BARBARA                          JALAMA   
44            TUU-5791      STANISLAUS                         PELICAN   
45          ANF-002517     LOS ANGELES   ASSIST (CROWN COMPLE 07/20/04   

   START_DATE        CONT_DATE ORIGIN_DPT BURNED_TOTAL VEG_TYPE         CAUSE  \
1    03/25/04         03/26/04       USFS          350       BT        ESCAPE   
2    04/09/04         04/12/04       USFS          301        T         HUMAN   
3    04/25/04         04/26/04        CDF        2,464       GB       VEHICLE   
4    05/02/04         05/03/04       MIL.        2,040       BT            UI   
5    05/02/04         05/07/04        CDF        8,831       BT  UNDETERMINED   
6    05/02/04         05/03/04        CDF          450       GB  UNDETERMINED   
7    05/03/04         05/03/04        CD

However, the table starts like this

In [17]:
display(largest_2004_1)

Unnamed: 0  Unnamed: 1                      Unnamed: 2  \
0             INC. # COUNTY         NaN                       FIRE NAME   
1   BDF-3917 SAN BERNARDINO         NaN                 UPPER SANTA ANA   
2         SHF-000443 TEHAMA         NaN           SHF ASSIST (HAMILTON)   
3      RRU-032913 RIVERSIDE         NaN                   PLEASURE FIRE   
4      MVU-003337 SAN DIEGO         NaN         ASSIST #31 (INDIA FIRE)   
5       RRU-35190 RIVERSIDE         NaN                           EAGLE   
6       RRU-35197 RIVERSIDE         NaN                         GAFFORD   
7       RRU-35567 RIVERSIDE         NaN                     SCHOOL INC.   
8      RRU-035517 RIVERSIDE         NaN                CERRITO FIRE 681   
9    SLU-3169 SANTA BARBARA         NaN             ASSIST #6 (CACHUMA)   
10        TUU-004886 TULARE         NaN                         FRAZIER   
11      RRU-38418 RIVERSIDE         NaN                      COTTONWOOD   
12        CA-PNF-379 PLUMAS         NaN                      COTTONWOOD   
13        TUU-005509 TULARE         NaN                           STAGE   
14          MMU-8972 MERCED         NaN                          BASALT   
15          NEU-009119 YUBA         NaN                           BEALE   
16   SCU-003038 SANTA CLARA         NaN                        PRIDMORE   
17   SBC-4032 SANTA BARBARA         NaN                         GAVIOTA   
18        TUU-006407 TULARE         NaN                          CASTLE   
19          SNF-1029 MADERA         NaN                          SOURCE   
20          SQF-1461 TULARE         NaN                            CRAG   
21            TUU-6885 KERN         NaN            KERN #11 (WEST FIRE)   
22        AEU-010202 AMADOR         NaN                     MOKE ASSIST   
23          LMU-1852 LASSEN         NaN                       SKEDADDLE   
24     BDU-6547 LOS ANGELES         NaN  ASSIST ANF (INTERSTATE06/26/04   
25        FKU-007654 FRESNO         NaN                        FKU-7654   
26       MMU-018987MARIPOSA         NaN                          MEADOW   
27          LNP-2017 SHASTA         NaN                       BLUFF WFU   
28      BEU-002411 MONTEREY         NaN            CABEU002411 (CHULAR)   
29      TGU-003837 TUOLUMNE         NaN                        BLACK, H   
30       MVU-5269 SAN DIEGO         NaN                          WARNER   
31     RRU-055439 RIVERSIDE         NaN                        VERBENIA   
32      SCU-3829 SANTA CRUZ         NaN                   CORRAL HOLLOW   
33         FKU-08513 FRESNO         NaN                           GATOS   
34        NEU-011956 PLACER         NaN                          SUNSET   
35   SCU-003929 SANTA CLARA         NaN                          SILVER   
36     BDU-7509 LOS ANGELES         NaN                            PINE   
37       MVU-5659 SAN DIEGO         NaN                       MATAGUARY   
38     RRU-056039 RIVERSIDE         NaN                        LAKEVIEW   
39          TUU-008451 KERN         NaN                      ROCKYKRN#1   
40      RRU-57236 RIVERSIDE         NaN                  STATE (MELTON)   
41     BDU-7498 LOS ANGELES         NaN               ASSIST (FOOTHILL)   
42        FKU-008892 FRESNO         NaN                         MENDOTA   
43   SBC-5251 SANTA BARBARA         NaN                          JALAMA   
44      TUU-5791 STANISLAUS         NaN                         PELICAN   
45   ANF-002517 LOS ANGELES         NaN   ASSIST (CROWN COMPLE 07/20/04   

                        DATE  Unnamed: 4     ACRES BURNED  VEG.    Unnamed: 7  \
0            START CONT. DPA         NaN  CDF OTHER TOTAL  TYPE         CAUSE   
1     03/25/04 03/26/04 USFS         NaN          350 350    BT        ESCAPE   
2     04/09/04 04/12/04 USFS         NaN          301 301     T         HUMAN   
3      04/25/04 04/26/04 CDF         NaN      2,464 2,464    GB       VEHICLE   
4     05/02/04 05/03/04 MIL.         NaN      2,040 2,040    BT            UI   
5    

### 1. Separating columns stuck together

The following columns are stuck together
* `INCIDENT #` & `COUNTY/UNIT`
* `START_DATE` & `CONT_DATE` & `ORIGIN_DPT`
* `CDF_BURNED` & `OTHER_BURNED` & `BURNED_TOTAL`
* both `STRUCTURE` cols & both `FATALITIES` cols

Some notes about the code:
* You can call `robust_col_sep` on the 'same' column multiple times
* Setting new column names is particularly useful here for applying column separator multiple times on the 'same' column
* In the last function call, notice the front keyword being used, we know that total will always be the third or last value in the grouped totals, therefore we set front to false in order to isolate the element in the back

In [20]:
clean_2004_1 = robust_col_sep(largest_2004_1, 0, ['0','1'])
clean_2004_1 = robust_col_sep(clean_2004_1, 4, ['4','5'])
clean_2004_1 = robust_col_sep(clean_2004_1, 5, ['5','6'])
clean_2004_1 = robust_col_sep(clean_2004_1, 8, ['8','9'], front=False)
clean_2004_1 = robust_col_sep(clean_2004_1, 12, ['12', '13'])
clean_2004_1 = robust_col_sep(clean_2004_1, 13, ['13', '14'])
clean_2004_1 = robust_col_sep(clean_2004_1, 14, ['14', '15'])
clean_2004_1

0               1  Unnamed: 1  \
0                 INC.        # COUNTY         NaN   
1             BDF-3917  SAN BERNARDINO         NaN   
2           SHF-000443          TEHAMA         NaN   
3           RRU-032913       RIVERSIDE         NaN   
4           MVU-003337       SAN DIEGO         NaN   
5            RRU-35190       RIVERSIDE         NaN   
6            RRU-35197       RIVERSIDE         NaN   
7            RRU-35567       RIVERSIDE         NaN   
8           RRU-035517       RIVERSIDE         NaN   
9             SLU-3169   SANTA BARBARA         NaN   
10          TUU-004886          TULARE         NaN   
11           RRU-38418       RIVERSIDE         NaN   
12          CA-PNF-379          PLUMAS         NaN   
13          TUU-005509          TULARE         NaN   
14            MMU-8972          MERCED         NaN   
15          NEU-009119            YUBA         NaN   
16          SCU-003038     SANTA CLARA         NaN   
17            SBC-4032   SANTA BARBARA         NaN   
18          TUU-006407          TULARE         NaN   
19            SNF-1029          MADERA         NaN   
20            SQF-1461          TULARE         NaN   
21            TUU-6885            KERN         NaN   
22          AEU-010202          AMADOR         NaN   
23            LMU-1852          LASSEN         NaN   
24            BDU-6547     LOS ANGELES         NaN   
25          FKU-007654          FRESNO         NaN   
26  MMU-018987MARIPOSA                         NaN   
27            LNP-2017          SHASTA         NaN   
28          BEU-002411        MONTEREY         NaN   
29          TGU-003837        TUOLUMNE         NaN   
30            MVU-5269       SAN DIEGO         NaN   
31          RRU-055439       RIVERSIDE         NaN   
32            SCU-3829      SANTA CRUZ         NaN   
33           FKU-08513          FRESNO         NaN   
34          NEU-011956          PLACER         NaN   
35          SCU-003929     SANTA CLARA         NaN   
36            BDU-7509     LOS ANGELES         NaN   
37            MVU-5659       SAN DIEGO         NaN   
38          RRU-056039       RIVERSIDE         NaN   
39          TUU-008451            KERN         NaN   
40           RRU-57236       RIVERSIDE         NaN   
41            BDU-7498     LOS ANGELES         NaN   
42          FKU-008892          FRESNO         NaN   
43            SBC-5251   SANTA BARBARA         NaN   
44            TUU-5791      STANISLAUS         NaN   
45          ANF-002517     LOS ANGELES         NaN   

                        Unnamed: 2         4                5     6  \
0                        FIRE NAME     START            CONT.   DPA   
1                  UPPER SANTA ANA  03/25/04         03/26/04  USFS   
2            SHF ASSIST (HAMILTON)  04/09/04         04/12/04  USFS   
3                    PLEASURE FIRE  04/25/04         04/26/04   CDF   
4          ASSIST #31 (INDIA FIRE)  05/02/04         05/03/04  MIL.   
5                            EAGLE  05/02/04         05/07/04   CDF   
6                          GAFFORD  05/02/04         05/03/04   CDF   
7                      SCHOOL INC.  05/03/04         05/03/04   CDF   
8                 CERRITO FIRE 681  05/03/04         05/07/04   CDF   
9              ASSIST #6 (CACHUMA)  05/03/04         05/07/04   LOC   
10                         FRAZIER  05/07/04         05/07/04   CDF   
11                      COTTONWOOD  05/12/04         05/15/04   CDF   
12                      COTTONWOOD  05/18/04         05/18/04  USFS   
13                           STAGE  05/21/04         05/21/04   CDF   
14                          BASALT  05/21/04         05/21/04   CDF   
15                           BEALE  06/03/04         06/03/04   CDF   
16                        PRIDMORE  06/04/04         06/04/04   CDF   
17                         GAVIOTA  06/05/04         06/11/04    CC   
18                          CASTLE  06/08/04         06/08/04   CDF   
19                          SOURCE  06/16/04         06/16/04 

### 2. & 3. Dropping other burned acres, NaN columns, and useless rows

In [21]:
clean_2004_1 = clean_2004_1.drop(0)
clean_2004_1 = clean_2004_1.drop(columns=clean_2004_1.columns[[2, 7, 8]])
clean_2004_1

0               1                      Unnamed: 2  \
1             BDF-3917  SAN BERNARDINO                 UPPER SANTA ANA   
2           SHF-000443          TEHAMA           SHF ASSIST (HAMILTON)   
3           RRU-032913       RIVERSIDE                   PLEASURE FIRE   
4           MVU-003337       SAN DIEGO         ASSIST #31 (INDIA FIRE)   
5            RRU-35190       RIVERSIDE                           EAGLE   
6            RRU-35197       RIVERSIDE                         GAFFORD   
7            RRU-35567       RIVERSIDE                     SCHOOL INC.   
8           RRU-035517       RIVERSIDE                CERRITO FIRE 681   
9             SLU-3169   SANTA BARBARA             ASSIST #6 (CACHUMA)   
10          TUU-004886          TULARE                         FRAZIER   
11           RRU-38418       RIVERSIDE                      COTTONWOOD   
12          CA-PNF-379          PLUMAS                      COTTONWOOD   
13          TUU-005509          TULARE                           STAGE   
14            MMU-8972          MERCED                          BASALT   
15          NEU-009119            YUBA                           BEALE   
16          SCU-003038     SANTA CLARA                        PRIDMORE   
17            SBC-4032   SANTA BARBARA                         GAVIOTA   
18          TUU-006407          TULARE                          CASTLE   
19            SNF-1029          MADERA                          SOURCE   
20            SQF-1461          TULARE                            CRAG   
21            TUU-6885            KERN            KERN #11 (WEST FIRE)   
22          AEU-010202          AMADOR                     MOKE ASSIST   
23            LMU-1852          LASSEN                       SKEDADDLE   
24            BDU-6547     LOS ANGELES  ASSIST ANF (INTERSTATE06/26/04   
25          FKU-007654          FRESNO                        FKU-7654   
26  MMU-018987MARIPOSA                                          MEADOW   
27            LNP-2017          SHASTA                       BLUFF WFU   
28          BEU-002411        MONTEREY            CABEU002411 (CHULAR)   
29          TGU-003837        TUOLUMNE                        BLACK, H   
30            MVU-5269       SAN DIEGO                          WARNER   
31          RRU-055439       RIVERSIDE                        VERBENIA   
32            SCU-3829      SANTA CRUZ                   CORRAL HOLLOW   
33           FKU-08513          FRESNO                           GATOS   
34          NEU-011956          PLACER                          SUNSET   
35          SCU-003929     SANTA CLARA                          SILVER   
36            BDU-7509     LOS ANGELES                            PINE   
37            MVU-5659       SAN DIEGO                       MATAGUARY   
38          RRU-056039       RIVERSIDE                        LAKEVIEW   
39          TUU-008451            KERN                      ROCKYKRN#1   
40           RRU-57236       RIVERSIDE                  STATE (MELTON)   
41            BDU-7498     LOS ANGELES               ASSIST (FOOTHILL)   
42          FKU-008892          FRESNO                         MENDOTA   
43            SBC-5251   SANTA BARBARA                          JALAMA   
44            TUU-5791      STANISLAUS                         PELICAN   
45          ANF-002517     LOS ANGELES   ASSIST (CROWN COMPLE 07/20/04   

           4                5     6       9 VEG.    Unnamed: 7  12 13 14 15  
1   03/25/04         03/26/04  USFS     350   BT        ESCAPE   1  0  0  0  
2   04/09/04         04/12/04  USFS     301    T         HUMAN   0  0  0  0  
3   04/25/04         04/26/04   CDF   2,464   GB       VEHICLE   9  0  0  0  
4   05/02/04         05/03/04  MIL.   2,040   BT            UI   0  0  0  0  
5   05/02/04         05/07/04   CDF   8,831   BT  UNDETERMINED  41  0  0  0  
6   05/02/04         05/03/04   CDF     450   GB  UNDETERMINED   0  0  0  0  
7   05/03/04         05/03/04   CDF     377   BT        DEBRIS   0  0 

### 4. Add correct column names

In [22]:
clean_2004_1.columns = ROWS_N
clean_2004_1

INCIDENT #     COUNTY/UNIT                       FIRE_NAME  \
1             BDF-3917  SAN BERNARDINO                 UPPER SANTA ANA   
2           SHF-000443          TEHAMA           SHF ASSIST (HAMILTON)   
3           RRU-032913       RIVERSIDE                   PLEASURE FIRE   
4           MVU-003337       SAN DIEGO         ASSIST #31 (INDIA FIRE)   
5            RRU-35190       RIVERSIDE                           EAGLE   
6            RRU-35197       RIVERSIDE                         GAFFORD   
7            RRU-35567       RIVERSIDE                     SCHOOL INC.   
8           RRU-035517       RIVERSIDE                CERRITO FIRE 681   
9             SLU-3169   SANTA BARBARA             ASSIST #6 (CACHUMA)   
10          TUU-004886          TULARE                         FRAZIER   
11           RRU-38418       RIVERSIDE                      COTTONWOOD   
12          CA-PNF-379          PLUMAS                      COTTONWOOD   
13          TUU-005509          TULARE                           STAGE   
14            MMU-8972          MERCED                          BASALT   
15          NEU-009119            YUBA                           BEALE   
16          SCU-003038     SANTA CLARA                        PRIDMORE   
17            SBC-4032   SANTA BARBARA                         GAVIOTA   
18          TUU-006407          TULARE                          CASTLE   
19            SNF-1029          MADERA                          SOURCE   
20            SQF-1461          TULARE                            CRAG   
21            TUU-6885            KERN            KERN #11 (WEST FIRE)   
22          AEU-010202          AMADOR                     MOKE ASSIST   
23            LMU-1852          LASSEN                       SKEDADDLE   
24            BDU-6547     LOS ANGELES  ASSIST ANF (INTERSTATE06/26/04   
25          FKU-007654          FRESNO                        FKU-7654   
26  MMU-018987MARIPOSA                                          MEADOW   
27            LNP-2017          SHASTA                       BLUFF WFU   
28          BEU-002411        MONTEREY            CABEU002411 (CHULAR)   
29          TGU-003837        TUOLUMNE                        BLACK, H   
30            MVU-5269       SAN DIEGO                          WARNER   
31          RRU-055439       RIVERSIDE                        VERBENIA   
32            SCU-3829      SANTA CRUZ                   CORRAL HOLLOW   
33           FKU-08513          FRESNO                           GATOS   
34          NEU-011956          PLACER                          SUNSET   
35          SCU-003929     SANTA CLARA                          SILVER   
36            BDU-7509     LOS ANGELES                            PINE   
37            MVU-5659       SAN DIEGO                       MATAGUARY   
38          RRU-056039       RIVERSIDE                        LAKEVIEW   
39          TUU-008451            KERN                      ROCKYKRN#1   
40           RRU-57236       RIVERSIDE                  STATE (MELTON)   
41            BDU-7498     LOS ANGELES               ASSIST (FOOTHILL)   
42          FKU-008892          FRESNO                         MENDOTA   
43            SBC-5251   SANTA BARBARA                          JALAMA   
44            TUU-5791      STANISLAUS                         PELICAN   
45          ANF-002517     LOS ANGELES   ASSIST (CROWN COMPLE 07/20/04   

   START_DATE        CONT_DATE ORIGIN_DPT BURNED_TOTAL VEG_TYPE         CAUSE  \
1    03/25/04         03/26/04       USFS          350       BT        ESCAPE   
2    04/09/04         04/12/04       USFS          301        T         HUMAN   
3    04/25/04         04/26/04        CDF        2,464       GB       VEHICLE   
4    05/02/04         05/03/04       MIL.        2,040       BT            UI   
5    05/02/04         05/07/04        CDF        8,831       BT  UNDETERMINED   
6    05/02/04         05/03/04        CDF          450       GB  UNDETERMINED   
7    05/03/04         05/03/04        CD

### Artifact Separation and COUNTY/UNIT and FIRE_NAME separation

#### Artifacts
* regex separator? most artifacts happen with numbers and letters

#### COUNTY/UNIT and FIRE_NAME separation
* make a list of actual COUNTY/UNIT names and see if combinations of the combined columns match any values we have

## END TUTORIAL

In [12]:
#combined dataset, this will grow as more datasets are cleaned
clean_df = pd.concat([clean_2000_1, clean_2000_2,
                      clean_2001_1, clean_2001_2,
                      clean_2002_1, clean_2002_2, clean_2002_3,
                      clean_2003_1, clean_2003_2, clean_2003_2,
                      clean_2004_1, clean_2004_2], ignore_index=True, sort=False)
clean_df['START_DATE'] = clean_df.START_DATE.astype(str).apply(datetime_convert_pdf)
clean_df['CONT_DATE'] = clean_df.CONT_DATE.astype(str).apply(datetime_convert_pdf)
clean_df['BURNED_TOTAL'] = clean_df.BURNED_TOTAL.apply(int_convert)
clean_df.head()

INCIDENT #            COUNTY/UNIT            FIRE_NAME  \
0              TUU-5954                 Tulare            Richgrove   
1              MVU-4431  San Diego (exp002 in)              Barrett   
2              MMU-9447               Mariposa              Granite   
3              BDU-5166         San Bernardino               Citrus   
4              BDU-5318         San Bernardino      Assist (Morgan)   
5              TUU-6521                 Tulare              Success   
6              LMU-1817                 Lassen                 Goat   
7              SKU-2492               Siskiyou  Bark KNF Assist #12   
8              BBD-1891        San Luis Obispo                Shell   
9              MVU-5080              San Diego   Assist #18 Buckman   
10             BDU-5882         Inyo fire #168           Happy Camp   
11             SQF-1883                 Tulare               Manter   
12             SCU-2756            Santa Clara            Patterson   
13             SKU-2574               Siskiyou            Corp. #13   
14             LNU-4423              Lake-Napa               Morgan   
15            RRU-48329              Riverside     State (Pachenga)   
16             BDU-6114            Mono County            Golden II   
17             TUU-7396   Kern County (Tulare)               Chance   
18             TUU-7440   Kern County (Tulare)                 King   
19             LMU-2017         Nevada (State)              Redrock   
20             LMU-2018         Nevada (State)           Arrowcreek   
21            TUU-73431   Kern County (Tulare)                 Peak   
22            VNC-16413                Ventura            Telegraph   
23             FKU-9369                 Fresno             FKU 9369   
24            RRU-58237    Riverside fire#1454          Black Ranch   
25            ORC-50963                 Orange             TRW Fire   
26             SBC-5687          Santa Barbara               Harris   
27             MVU-6514              San Diego                Lyons   
28             BDU-8329            Los Angeles             Paradise   
29             BDU-7525  Los Angeles LAC162392                Yucca   
..                  ...                    ...                  ...   
409          TCU-006440              CALAVERAS    HWY 4 #2 (COPPER)   
410            SCU-4410            SANTA CLARA              KINCAID   
411           NEU-13741                 PLACER              STEVENS   
412          MEU-006378              MENDOCINO                CREEK   
413          BTU-009671                  BUTTE               OREGON   
414            BDU-9291            LOS ANGELES        EAST ANF 2861   
415          TCU-006563              CALAVERAS                 HUNT   
416          TCU-006534               TUOLUMNE                EARLY   
417          SHU-006278                 SHASTA                 BEAR   
418          TUU-009943                 TULARE                 DEEP   
419            LMU-2930                 LASSEN     NOD BBT4 (SNAKE)   
420            NOD-2946                 LASSEN               STONEY   
421          LMU-002930                 LASSEN     LGT#60 (BELFAST)   
422          SHU-006398                 SHASTA               FRENCH   
423            LNU-6125                 SOLANO           MIX (VACA)   
424          LNU-006346            LAKE/SONOMA                 LONG   
425  MMU-016500MARIPOSA                                        BEAR   
426          LNU-006644                   LAKE              GEYSERS   
427          TCU-007413              CALAVERAS             PATTISON   
428          LNU-006675                   NAPA               CEMENT   
429          AEU-015503             SACRAMENTO                SCOTT   
430            BDU-9520         SAN BERNARDINO     ASSIST - RUNAWAY   
431            LNU-6821                  MARIN              DOLCINI   
432           TUU-11429                   KERN                CHINA   
433  MMU-017290MARIPOSA              

In [ ]:
joined = clean_df.merge(fires, left_on=['BURNED_TOTAL', 'START_DATE', 'CONT_DATE'], right_on=['FIRE_SIZE', 'START_DATE', 'END_DATE'])
joined.head()

## Future Sections

While not data sets are added (yet!), the coding for other sections can start since these are the final columns kept, as more data is added, the code should mould to the new data

## EDA

In [ ]:
#folium map, to view open map.html
m = folium.Map(location = [36, -119])
lats = joined.LATITUDE
longs = joined.LONGITUDE
for i in np.arange(len(lats)):
    folium.Marker(location=[lats[i], longs[i]], icon=folium.Icon(color='red', icon='fire')).add_to(m)
m.save('map.html')